In [40]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [41]:
path = 'Data/'

In [42]:
df = pd.read_sas(path+'LLCP2022.xpt')
df

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_SMOKGRP,_LCSREC,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1.0,1.0,b'02032022',b'02',b'03',b'2022',1100.0,b'2022000001',2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,2.0,2.0
1,1.0,1.0,b'02042022',b'02',b'04',b'2022',1100.0,b'2022000002',2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,2.0,2.0,2.0
2,1.0,1.0,b'02022022',b'02',b'02',b'2022',1100.0,b'2022000003',2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,NaN,NaN,2.0
3,1.0,1.0,b'02032022',b'02',b'03',b'2022',1100.0,b'2022000004',2.022000e+09,1.0,...,3.0,2.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,9.0,9.0,2.0
4,1.0,1.0,b'02022022',b'02',b'02',b'2022',1100.0,b'2022000005',2.022000e+09,1.0,...,4.0,NaN,1.0,1.000000e+01,1.0,1.400000e+02,1.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,78.0,11.0,b'12192022',b'12',b'19',b'2022',1100.0,b'2022001527',2.022002e+09,NaN,...,4.0,NaN,7.0,9.000000e+02,9.0,9.990000e+04,9.0,NaN,NaN,1.0
445128,78.0,11.0,b'12212022',b'12',b'21',b'2022',1100.0,b'2022001528',2.022002e+09,NaN,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,NaN,NaN,1.0
445129,78.0,11.0,b'11292022',b'11',b'29',b'2022',1100.0,b'2022001529',2.022002e+09,NaN,...,1.0,NaN,7.0,9.000000e+02,9.0,9.990000e+04,9.0,2.0,2.0,2.0
445130,78.0,11.0,b'12082022',b'12',b'08',b'2022',1100.0,b'2022001530',2.022002e+09,NaN,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,1.0


In [43]:
df.shape

(445132, 328)

In [44]:
# # keep only fully completed interviews
# df = df[df['DISPCODE']==1100]
# df

In [45]:
columns = df.columns

In [46]:
# df.to_csv(path+'brfss2022.csv')

In [47]:
selected_cols = [
    'SEXVAR',
    'GENHLTH',
    'PHYSHLTH',
    'MENTHLTH',
#     'POORHLTH',
    'PERSDOC3',
    'MEDCOST1',
    'CHECKUP1',
    'EXERANY2',
    'SLEPTIM1',
    'CVDINFR4',
    'CVDCRHD4',
    'CVDSTRK3',
    'ADDEPEV3',
    'DIABETE4',
    'MARITAL',
    'EDUCA',
    'EMPLOY1',
    'CHILDREN',
    'INCOME3',
#     'PREGNANT',
    'WEIGHT2',
    'HEIGHT3',
#     'DECIDE',
    'DIFFWALK',
    'SMOKE100',
#     'SMOKDAY2',
#     'USENOW3',
#     'ECIGNOW2',
    'ALCDAY4',
#     'AVEDRNK3',
    'PDIABTS1',
    'PREDIAB2',
#     'DIABTYPE',
#     'INSULIN1',
#     'CHKHEMO3',
#     'EYEEXAM1',
#     'DIABEYE1',
#     'DIABEDU1',
#     'FEETSORE',
#     'ACEDIVRC',
#     'ACEPUNCH',
#     'ACEHURT1',
#     'ACESWEAR',
#     'ACEADNED',
#     'LSATISFY',
#     'EMTSUPRT',
#     'SDHISOLT',
#     'SDHTRNSP',
#     'SDHSTRE1',
#     'LASTSMK2',
#     'BIRTHSEX',
#     '_METSTAT',
#     '_URBSTAT',
    '_IMPRACE',
    '_RFHLTH',
    '_PHYS14D',
    '_MENT14D',
    '_HLTHPLN',
]

In [48]:
df = df[selected_cols]
df = df.dropna()


In [50]:
col_mapping = {
    'SEXVAR':'sex',
#     'GENHLTH',
#     'PHYSHLTH',
#     'MENTHLTH',
#     'PERSDOC3',
#     'MEDCOST1',
#     'CHECKUP1',
#     'EXERANY2',
#     'SLEPTIM1',
#     'CVDINFR4',
#     'CVDCRHD4',
#     'CVDSTRK3',
#     'ADDEPEV3',
#     'DIABETE4',
#     'MARITAL',
#     'EDUCA',
#     'EMPLOY1',
#     'CHILDREN',
#     'INCOME3',
#     'WEIGHT2',
#     'HEIGHT3',
#     'DIFFWALK',
#     'SMOKE100',
#     'ALCDAY4',
#     'PDIABTS1',
#     'PREDIAB2',
#     '_IMPRACE',
#     '_RFHLTH',
#     '_PHYS14D',
#     '_MENT14D',
#     '_HLTHPLN',
    

}

In [51]:
df.rename(columns=col_mapping, inplace=True)

In [52]:
df

,sex,GENHLTH,PHYSHLTH,MENTHLTH,PERSDOC3,MEDCOST1,CHECKUP1,EXERANY2,SLEPTIM1,CVDINFR4,...,DIFFWALK,SMOKE100,ALCDAY4,PDIABTS1,PREDIAB2,_IMPRACE,_RFHLTH,_PHYS14D,_MENT14D,_HLTHPLN
327,2.0,2.0,88.0,5.0,1.0,2.0,1.0,1.0,6.0,2.0,...,2.0,2.0,105.0,2.0,3.0,1.0,1.0,1.0,2.0,1.0
328,2.0,2.0,88.0,88.0,2.0,2.0,1.0,2.0,9.0,2.0,...,1.0,2.0,888.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0
330,2.0,3.0,88.0,88.0,1.0,2.0,1.0,2.0,5.0,2.0,...,1.0,2.0,888.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0
333,1.0,1.0,2.0,88.0,1.0,2.0,1.0,1.0,6.0,2.0,...,1.0,2.0,101.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0
334,2.0,3.0,88.0,88.0,2.0,2.0,1.0,1.0,8.0,2.0,...,2.0,1.0,888.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,2.0,3.0,88.0,3.0,3.0,2.0,2.0,1.0,6.0,2.0,...,2.0,2.0,777.0,7.0,3.0,2.0,1.0,1.0,2.0,1.0
445128,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,7.0,2.0,...,2.0,2.0,888.0,1.0,3.0,2.0,1.0,2.0,2.0,1.0
445129,2.0,5.0,30.0,30.0,3.0,1.0,4.0,2.0,5.0,2.0,...,2.0,1.0,777.0,7.0,3.0,2.0,2.0,3.0,3.0,2.0
445130,1.0,2.0,88.0,88.0,2.0,2.0,1.0,2.0,5.0,1.0,...,2.0,2.0,888.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0


TODO: 
1. rename columns from dictionary
2. change data
3. change data types
4. save csv file
5. upload to kaggle